In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold
from keras import models, regularizers
from keras import layers
from numpy.random import seed
import tensorflow as tf

seed(1610)
tf.random.set_seed(1610)

In [2]:
train_data = pd.read_excel('Molecules.xlsx')

train_data = train_data.sample(frac=1)
# train_data.head()
# train_data.dtypes


In [3]:
X = train_data.iloc[:, 4:131]
Y = train_data.iloc[:,0]

X = X.values
Y = Y.values

In [4]:
kf = KFold(n_splits = 8, shuffle=True)

In [5]:
train_sets =[]
test_sets = []
for train_index, test_index in kf.split(X, Y):
    train_sets.append(train_index)
    test_sets.append(test_index)


In [6]:
model1 = models.Sequential()
model1.add(layers.Dense(256, activation='relu'))
model1.add(layers.Dense(1, activation='sigmoid'))
model1.compile(optimizer='Adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model11 = models.Sequential()
model11.add(layers.Dense(128, activation='relu'))
model11.add(layers.Dense(1, activation='sigmoid'))
model11.compile(optimizer='Adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model2 = models.Sequential()
model2.add(layers.Dense(256, activation='relu')) #, kernel_regularizer = regularizers.l2(0.1)))
model2.add(layers.Dense(256, activation='relu')) #, kernel_regularizer = regularizers.l2(0.1)))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer='Adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model21 = models.Sequential()
model21.add(layers.Dense(128, activation='relu')) #, kernel_regularizer = regularizers.l2(0.1)))
model21.add(layers.Dense(128, activation='relu')) #, kernel_regularizer = regularizers.l2(0.1)))
model21.add(layers.Dense(1, activation='sigmoid'))
model21.compile(optimizer='Adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model3 = models.Sequential()
model3.add(layers.Dense(256, activation='relu')) #, kernel_regularizer = regularizers.l2(0.1)))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.compile(optimizer='Adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model31 = models.Sequential()
model31.add(layers.Dense(128, activation='relu')) #, kernel_regularizer = regularizers.l2(0.1)))
model31.add(layers.Dropout(0.5))
model31.add(layers.Dense(1, activation='sigmoid'))
model31.compile(optimizer='Adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model4 = models.Sequential()
model4.add(layers.Dense(256, activation='relu'))
model4.add(layers.Dropout(0.5))
model4.add(layers.Dense(256, activation='relu'))
model4.add(layers.Dropout(0.5))
model4.add(layers.Dense(1, activation='sigmoid'))
model4.compile(optimizer='Adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model41 = models.Sequential()
model41.add(layers.Dense(128, activation='relu'))
model41.add(layers.Dropout(0.5))
model41.add(layers.Dense(128, activation='relu'))
model41.add(layers.Dropout(0.5))
model41.add(layers.Dense(1, activation='sigmoid'))
model41.compile(optimizer='Adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

Accuracy = []
AUC = []
min_train_error = []
models = []
best_models = []

for i in range(8):
    X_train = X[train_sets[i],:]
    Y_train = Y[train_sets[i]]

    X_test = X[test_sets[i],:]
    Y_test = Y[test_sets[i]]

    mean = X_train.mean(axis=0)
    X_train -= mean
    std = X_train.std(axis=0)
    X_train /= std

    X_test -= mean
    X_test /= std

    min_train_error = []

    history1 = model1.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 0)
    train_error1 = history1.history['accuracy']
    min_train_error.append(train_error1[99])
    models.append(model1)

    history11 = model11.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 0)
    train_error11 = history11.history['accuracy']
    min_train_error.append(train_error11[99])
    models.append(model11)

    history2 = model2.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 0)
    train_error2 = history2.history['accuracy']
    min_train_error.append(train_error2[99])
    models.append(model2)

    history21 = model21.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 0)
    train_error21 = history21.history['accuracy']
    min_train_error.append(train_error21[99])
    models.append(model21)

    history3 = model3.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 0)
    train_error3 = history3.history['accuracy']
    min_train_error.append(train_error3[99])
    models.append(model3)

    history31 = model31.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 0)
    train_error31 = history31.history['accuracy']
    min_train_error.append(train_error31[99])
    models.append(model31)

    history4 = model4.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 0)
    train_error4 = history4.history['accuracy']
    min_train_error.append(train_error4[99])
    models.append(model4)

    history41 = model41.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 0)
    train_error41 = history41.history['accuracy']
    min_train_error.append(train_error41[99])
    models.append(model41)

    best_model = []
    best_model =  np.argmin(min_train_error)

    best_models.append('model'+str(best_model))

    Yhat = models[best_model].predict(X_test)

    Yhat_01 = np.where(Yhat > 0.5, 1, 0)
    conf = confusion_matrix(Y_test, Yhat_01)
    Accuracy.append(sum(conf[i][i] for i in range(2))/sum(conf[i][j] for i in range(2) for j in range(2)))

    AUC.append(roc_auc_score(Y_test, Yhat))

In [7]:
print(best_models)

['model3', 'model7', 'model3', 'model5', 'model3', 'model5', 'model2', 'model3']


In [8]:
print("Accuracy: mean: ", np.mean(Accuracy), "sd: ", np.std(Accuracy))

Accuracy: mean:  0.5108695652173914 sd:  0.02300344925837314


In [9]:
print("AUC: mean: ", np.mean(AUC), "sd: ", np.std(AUC))

AUC: mean:  0.5165956834330954 sd:  0.03208902793508766
